In [ ]:
from pathlib import Path

import pandas as pd

from ttsim import main

GETTSIM_ROOT = Path.cwd() / "src" / "_gettsim"

inputs_df = pd.DataFrame(
    {
        "age": [30, 30, 10],
        "working_hours": [35, 35, 0],
        "disability_grade": [0, 0, 0],
    }
)

# Prototypes of GETTSIM's new interface

[GEP 7](https://gettsim--855.org.readthedocs.build/en/855/geps/gep-07.html) discusses
the principles of the new interface. This notebook demonstrates two candidates for
GETTIM's new interface. We would like to get your feedback on which one you prefer.

In this notebook, we compute income taxes and social security contributions for example
data.

## Setup

This notebook requires to have GETTSIM installed in its current development version.

To do this:
1. Clone the GETTSIM repository.
2. Install the [pixi package manager](https://pixi.sh/latest/) on your system.
3. `cd` into the GETTSIM repository and run `git checkout inputs-for-main`.
4. Run `pixi run jupyter-notebook` and select the `interface-prototype.ipynb` notebook.

If you have trouble with the setup, please reach out.

## Creating the Data

First, we create some example data. Here, we use a pandas DataFrame with column names
that are different from the ones GETTSIM expects.

In [ ]:
# Some example data as a standard pandas DataFrame
data = pd.DataFrame(
    {
        "age": [30, 30, 10],
        "working_hours": [35, 35, 0],
        "disability_grade": [0, 0, 0],
        "birth_year": [1995, 1995, 2015],
        "hh_id": [0, 0, 0],
        "p_id": [0, 1, 2],
        "east_germany": [False, False, False],
        "self_employed": [False, False, False],
        "income_from_self_employment": [0, 0, 0],
        "income_from_rent": [0, 0, 0],
        "income_from_employment": [5000, 4000, 0],
        "income_from_forest_and_agriculture": [0, 0, 0],
        "income_from_capital": [500, 0, 0],
        "income_from_other": [0, 0, 0],
        "pension_income": [0, 0, 0],
        "contribution_to_social_security": [0, 0, 0],
        "childcare_expenses": [0, 0, 0],
        "person_that_pays_childcare_expenses": [-1, -1, 0],
        "joint_taxation": [True, True, False],
        "amount_private_pension_income": [0, 0, 0],
        "contribution_private_health_insurance": [0, 0, 0],
        "has_children": [True, True, False],
        "single_parent": [False, False, False],
        "is_child": [False, False, True],
        "spouse_id": [1, 0, -1],
        "parent_id_1": [-1, -1, 0],
        "parent_id_2": [-1, -1, 1],
        "in_training": [False, False, False],
        "id_recipient_child_allowance": [-1, -1, 0],
        "wohngeld": [0, 0, 0],
        "kinderzuschlag": [0, 0, 0],
        "elterngeld": [0, 0, 0],
        "alg1": [0, 0, 0],
        "old_age_pension_income": [0, 0, 0],
        "bürgergeld": [0, 0, 0],
    }
)

The first step in GETTSIM's new workflow is to define the targets you're interested in.
The keys of the nested dictionary below are the paths GETTSIM will use as targets. For
instance, via the keys `einkommensteuer` and `betrag_m_sn`, we request the income tax as
a target.

The values on the lowest level of the dictionaries will be used as the column names of
the resulting DataFrame. Here, `income_tax_y` will be the name of the column in the
resulting DataFrame that contains the income tax.

In [ ]:
TARGETS_TREE = {
    "einkommensteuer": {"betrag_y_sn": "income_tax_y"},
    "sozialversicherung": {
        "pflege": {
            "beitrag": {
                "betrag_versicherter_m": "long_term_care_insurance_contribution_m"
            }
        },
        "kranken": {
            "beitrag": {"betrag_versicherter_m": "health_insurance_contribution_m"}
        },
        "rente": {
            "beitrag": {"betrag_versicherter_m": "pension_insurance_contribution_m"}
        },
        "arbeitslosen": {
            "beitrag": {
                "betrag_versicherter_m": "unemployment_insurance_contribution_m"
            }
        },
    },
}

Next, we need to map our input data to the structure GETTSIM expects. For this, we
create a template for the inputs GETTSIM expects to calculate the targets specified
above.

In [ ]:
GETTSIM_ROOT

In [ ]:
input_template = main(
    inputs={
        "date_str": "2025-01-01",
        "targets__tree": TARGETS_TREE,
        "orig_policy_objects__root": GETTSIM_ROOT,  # don't worry about this, will be gone in the future
    },
    output_names=["templates__input_data_dtypes"],
)["templates__input_data_dtypes"]

In [ ]:
policy_inputs = {
    "alter": [30, 30, 10],
    "arbeitsstunden_w": [35, 35, 0],
    "behinderungsgrad": [0, 0, 0],
    "geburtsjahr": [1995, 1995, 2015],
    "hh_id": [0, 0, 0],
    "p_id": [0, 1, 2],
    "wohnort_ost": [False, False, False],
    "einkommensteuer": {
        "einkünfte": {
            "ist_selbstständig": [False, False, False],
            "aus_gewerbebetrieb": {"betrag_m": [0, 0, 0]},
            "aus_vermietung_und_verpachtung": {"betrag_m": [0, 0, 0]},
            "aus_nichtselbstständiger_arbeit": {"bruttolohn_m": [5000.0, 4000.0, 0]},
            "aus_forst_und_landwirtschaft": {"betrag_m": [0, 0, 0]},
            "aus_selbstständiger_arbeit": {"betrag_m": [0, 0, 0]},
            "aus_kapitalvermögen": {"kapitalerträge_m": [500.0, 0.0, 0]},
            "sonstige": {"ohne_renten_m": [0, 0, 0], "renteneinkünfte_m": [0, 0, 0]},
        },
        "abzüge": {
            "beitrag_private_rentenversicherung_m": [0, 0, 0],
            "betreuungskosten_m": [0.0, 0.0, 100.0],
            "p_id_betreuungskostenträger": [-1, -1, 0],
        },
        "gemeinsam_veranlagt": [True, True, False],
    },
    "sozialversicherung": {
        "rente": {"private_rente_betrag_m": [0, 0, 0]},
        "kranken": {"beitrag": {"privat_versichert": [False, False, False]}},
        "pflege": {"beitrag": {"hat_kinder": [True, True, False]}},
    },
    "familie": {
        "alleinerziehend": [False, False, False],
        "kind": [False, False, True],
        "p_id_ehepartner": [1, 0, -1],
        "p_id_elternteil_1": [-1, -1, 0],
        "p_id_elternteil_2": [-1, -1, 1],
    },
    "kindergeld": {
        "in_ausbildung": [False, False, False],
        "p_id_empfänger": [-1, -1, 0],
    },
}

policy_inputs_overriding_functions = {
    "wohngeld": {"betrag_m_wthh": [0, 0, 0]},
    "kinderzuschlag": {"betrag_m": [0, 0, 0]},
    "elterngeld": {"betrag_m": [0, 0, 0]},
    "sozialversicherung": {
        "arbeitslosen": {"betrag_m": [0.0, 0.0, 0.0]},
        "rente": {"altersrente": {"betrag_m": [0.0, 0.0, 0.0]}},
    },
    "arbeitslosengeld_2": {"betrag_m_bg": [0, 0, 0]},
}